In [ ]:
import numpy as np
import ioh

In [ ]:
ioh.ProblemClass.GRAPH.problems

In [ ]:
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

In [ ]:
print(max_cut.meta_data)
print(max_coverage.meta_data)

In [ ]:
nb_it = 10000

In [ ]:
x_mcov = cut = np.random.randint(0,2,max_coverage.meta_data.n_variables)
val = max_coverage(x_mcov)
# help(max_coverage.optimum)
for ind in ioh.ProblemClass.GRAPH.problems:
    p = ioh.get_problem(ind, problem_class=ioh.ProblemClass.GRAPH)
    if np.any(p.optimum.x):
        print(p.meta_data)

# 1/2-approximation for MaxCut

Proof in Bertrand Meyer's Advanced Algorithms lecture

In [ ]:
max_cut.reset()

In [ ]:
cut = np.random.randint(0,2,max_cut.meta_data.n_variables)
max_cut(cut)
print(max_cut.state)

# RLS

In [ ]:
import numpy as np

class RandomLocalSearch:
    'Random local search algorithm'
    def __init__(self, n: int):
        self.n: int = n
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        'For each it flip 1 bit chosen uar and keep the best of both'
        
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            index_to_flip = np.random.randint(0,problem.meta_data.n_variables)
            y = x
            y[index_to_flip] = 1 - y[index_to_flip]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [ ]:
max_coverage.reset()
r = RandomLocalSearch(nb_it)
r(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

In [ ]:
max_cut.reset()
r = RandomLocalSearch(nb_it)
r(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

# (1+1) EA 

In [ ]:
class EA_1p1:
    '(1+1)-EA'
    def __init__(self, n: int, p : float):
        self.n: int = n
        self.p: float = p
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            y = x.copy()
            for ind in range(len(x)):
                y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [ ]:
max_coverage.reset()
ea_11 = EA_1p1(nb_it, 1/max_coverage.meta_data.n_variables)
ea_11(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

In [ ]:
max_cut.reset()
ea_11 = EA_1p1(nb_it, np.log(max_cut.meta_data.n_variables)/max_cut.meta_data.n_variables)
ea_11(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

# (1+($\lambda, \lambda$)) EA 

In [ ]:
max_cut.reset()

In [ ]:
import random

class EA_1plamlam:
    '(1+(lam,lam))-EA'
    def __init__(self, n: int, lam : int, p : float, c : float):
        self.n: int = n
        self.lam: int = lam
        self.p: float = p
        self.c: float = c
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        length_x = problem.meta_data.n_variables
        x = np.random.randint(0,2,length_x)
        val = problem(x)
        for _ in range(self.n):
            L = np.random.binomial(length_x, self.p)
            xp = None
            best_val = None # we allow for worse values than x
            for _ in range(self.lam):
                bits_to_flip = random.sample(range(length_x), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 1 - new_xp[i] if np.random.binomial(1, self.p) else new_xp[i]
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            y = x.copy()
            best_val = None
            for _ in range(self.lam):
                new_y = x.copy()
                for i in range(length_x):
                    if np.random.binomial(1,self.c):
                        new_y[i] = xp[i]
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            if best_val > val:
                x = y
                val = best_val

In [ ]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

# First idea 
as dealing with submodular functions, we may want to select the arguments that make it grow "the most at first" --> as when adding more elements we face diminishing returns
so from a solution x -> we build lam xs turning off L bits of uar and keeping the best -> we then build lam ys by flipping on L' bits of the best xs uniformly at random and keeping the best y --> then we turn on each bit present in y and not in x

problem: solution x keeps growing?
how to stop that?

In [ ]:
class OffAndOn:
    def __init__(self, n: int, lam : int, p_off : float, p_on : float, c : float):
        self.n: int = n
        self.p_off: int = p_off
        self.p_on: int = p_on
        self.lam: float = lam
        self.c: bool = c
        
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.zeros(problem.meta_data.n_variables, dtype=int)
        val = problem(x)
        x_ = np.random.randint(0,2,problem.meta_data.n_variables)
        val_ = problem(x)
        if val_ > val:
            x = x_
        for _ in range(self.n):
            L = np.random.binomial(problem.meta_data.n_variables, self.p_off)
            xp = None
            best_val = None 
            for _ in range(self.lam):
                # mutation 1 : we turn off L bits 
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 0
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            L = np.random.binomial(problem.meta_data.n_variables, self.p_on)
            y = xp.copy()
            best_val = None
            for _ in range(self.lam):
                # mutation 2 : we turn on L bits
                new_y = x.copy()
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                for i in bits_to_flip:
                    new_y[i] = 1
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            # mix:
            # if bit in y and not in x then I turn it on 
            mix = [0] * problem.meta_data.n_variables
            for i in range(problem.meta_data.n_variables):
                pc = np.random.binomial(1, self.c)
                if y[i] and not x[i]:
                    mix[i] = 1 if not pc else 0
                else:
                    mix[i] = x[i] if not pc else y[i]
            val_new = problem(mix)
            if val_new > val:
                x = mix
                val = best_val
            

In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, False, lam/dim_pb)
offandon(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

In [ ]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True, lam/dim_pb)
offandon(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

# Second idea:

from x:
 - generate lam children by flipping bits randomly 
 - take the lam2 best and do the intersection --> those are the "best elements to keep"
 - combine x and the intersection

In [ ]:
class IntersectAndCombine:
    def __init__(self, n: int, lam: int, lam2: int, p: float, c: float, init_a_0 : bool):
        self.n: int = n
        self.p: int = p
        self.c: int = c
        self.lam: float = lam
        self.lam2: float = lam2
        self.init_a_0: float = init_a_0

    def generate_child(self, x):
        y = x.copy()
        for ind in range(len(x)):
            y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
        return y

    def intersection(self, x, y):
        return np.logical_and(x, y).astype(int)

    def union(self, x, y):
        return np.logical_or(x, y).astype(int)

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.init_a_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
            val = problem(x)
        else:
            x = np.random.randint(0, 2, problem.meta_data.n_variables)
            val = problem(x)
        for tour in range(self.n):
            values = []
            children = []
            for _ in range(self.lam):
                x_ = self.generate_child(x)
                values.append(problem(x_))
                children.append(x_)

            children = np.array(children)
            sorted_inds = np.argsort(values)[:self.lam2]
            children = children[sorted_inds]
            intersection = np.logical_and.reduce(children).astype(int) # intersection will be small

            mix = [1 if intersection[i] else (intersection[i] if np.random.binomial(
                1, self.c) else x[i]) for i in range(problem.meta_data.n_variables)]
            val_mix = problem(mix)

            union = self.union(mix.copy(),x.copy())
            val_u = problem(union)

            ind = np.argmax([val, val_mix, val_u])
            
            val = [val, val_mix, val_u][ind]
            x = [x, mix, union][ind]  
            if (ind != 0):
                self.p = max(10/problem.meta_data.n_variables, self.p / 1.01)
                print(tour, "update", self.p, "  ind ", ind) 

In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
intcomb = IntersectAndCombine(900, lam, 3, 1/4, 10/dim_pb, False)
intcomb(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

In [ ]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 20
intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
intcomb(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

# Third idea (QB):

from x:
 - generate lam children by flipping bits randomly 
 - take the lam2 best and build a prob distr proportionnally to their obj values (TODO: see how to integrate sol x)
 - "crossover"?

In [ ]:
class ProbDistr:
    def __init__(self, n: int, lam: int, p: float, c: float, init_a_0 : bool):
        self.n: int = n
        self.p: int = p
        self.c: int = c
        self.lam: float = lam
        self.init_a_0: float = init_a_0

    def generate_child(self, x):
        y = x.copy()
        for ind in range(len(x)):
            y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
        return y

    def get_new_from_children(self, x, children, values, dim):
        sum = np.sum(values)
        p = [v/sum for v in values]
        new_x = np.zeros(dim).astype(int)
        for ind in range(dim):
            assert(self.lam == len(children))
            chosen_child = np.random.choice(np.arange(0, self.lam), p=p) 
            new_x[ind] = children[chosen_child][ind]
        return new_x

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.init_a_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
            val = problem(x)
        else:
            x = np.random.randint(0, 2, problem.meta_data.n_variables)
            val = problem(x)
        for tour in range(self.n):
            values = []
            children = []
            for _ in range(self.lam):
                x_ = self.generate_child(x)
                values.append(problem(x_))
                children.append(x_)

            new_x = self.get_new_from_children(x, np.array(children), np.array(values), problem.meta_data.n_variables)
            val_new_x = problem(new_x)

            mix = [new_x[i] if np.random.binomial(1, self.c) else x[i] for i in range(problem.meta_data.n_variables)]
            val_mix = problem(mix)

            ind = np.argmax([val, val_mix, val_new_x])
            
            val = [val, val_mix, val_new_x][ind]
            x = [x, mix, new_x][ind]  
            if (ind != 0):
                self.p = max(10/problem.meta_data.n_variables, self.p / 1.01)
                print(tour, "update", self.p, "  ind ", ind) 

In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
pd = ProbDistr(1000, lam, 1/4, 10/dim_pb, False)
pd(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

In [47]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 10
pd = ProbDistr(1000, lam, 1/4, 10/dim_pb, True)
pd(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

0 update 0.24752475247524752   ind  1
1 update 0.2450740123517302   ind  1
2 update 0.2426475369819111   ind  1
3 update 0.24024508612070405   ind  1
10 update 0.2378664219016872   ind  1
397 update 0.23551130881355167   ind  1
471 update 0.23317951367678383   ind  1
793 update 0.23087080562057805   ind  1
860 update 0.22858495605997825   ind  1
863 update 0.2263217386732458   ind  1
903 update 0.22408092937945126   ind  1
977 update 0.22186230631628837   ind  1
992 update 0.2196656498181073   ind  1
12001 420.0


In [ ]:
def run_experiment(problem, algorithm, n_runs=5,s="problem:"):
    for run in range(n_runs):
        algorithm(problem)
        print(s,problem.state.evaluations, problem.state.y_unconstrained_best)
        problem.reset()

In [ ]:
import os

logger_on_off = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="test_data",       # in a folder named: 'my-experiment'
    algorithm_name="on_off",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

ioh.ProblemClass.GRAPH.problems
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_on_off)
    dim_pb = problem.meta_data.n_variables
    lam = 10
    offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True, lam/dim_pb)
    run_experiment(problem, offandon, n_runs=1)

logger_on_off.close()

In [ ]:
logger_int_comb = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="test_data_int_comb",       
    algorithm_name="int_comb",    
    store_positions=True               
)

logger_int_comb

print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)
for i in range(2100,2128):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_int_comb)
    dim_pb = max_coverage.meta_data.n_variables
    lam = 20
    intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
    run_experiment(problem, intcomb, n_runs=1)

logger_int_comb.close()

In [ ]:
logger_int_comb_max_cut = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="test_data_int_comb_max_cut",       
    algorithm_name="int_comb",    
    store_positions=True               
)

logger_int_comb_max_cut

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_int_comb_max_cut)
    dim_pb = max_coverage.meta_data.n_variables
    lam = 20
    intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
    run_experiment(problem, offandon, n_runs=1)

logger_int_comb_max_cut.close()

In [ ]:
# 2300-2304 pachwhiletravel
for i in range(2300,2305):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    dim_pb = problem.meta_data.n_variables
    lam = 20
    intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
    run_experiment(problem, intcomb, n_runs=1,s="intcomb:")
    problem.reset()
    ea = EA_1p1(nb_it, 1/dim_pb)
    run_experiment(problem, ea, n_runs=1,s="ea:")
    problem.reset()

In [ ]:
#maxinfluence
for i in range(2200,2205):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    dim_pb = problem.meta_data.n_variables
    lam = 20
    intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
    run_experiment(problem, intcomb, n_runs=1,s="intcomb:")
    problem.reset()
    ea = EA_1p1(nb_it, 1/dim_pb)
    run_experiment(problem, ea, n_runs=1,s="ea:")
    problem.reset()